In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold, chi2, f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [5]:
df = pd.read_csv("employment_income_dataset.csv")
df = df.dropna(how='any',axis=0) 

display(df.describe())
df.info()
df.head()

,employee_id,age,years_experience,training_hours,certifications_count,performance_rating,manager_feedback_score,work_life_balance_score,salary_satisfaction,annual_income_usd,bonus_percent,commute_time_minutes,overtime_hours_monthly,job_satisfaction_score
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.0000
mean,1000.500000,42.694000,19.557500,20.417500,1.067000,3.489000,5.368800,5.470550,5.500000,70351.517500,9.854700,30.087000,4.945000,4.4311
std,577.494589,12.614195,11.438799,9.910919,1.055494,0.852592,2.580672,2.570849,2.632458,19690.991216,5.813682,10.105914,2.271245,2.1683
min,1.000000,21.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,30000.000000,0.000000,0.000000,0.000000,1.0000
25%,500.750000,32.000000,10.000000,13.000000,0.000000,2.800000,3.100000,3.300000,3.200000,56213.750000,4.700000,23.000000,3.000000,2.8000
50%,1000.500000,43.000000,20.000000,21.000000,1.000000,3.500000,5.400000,5.500000,5.500000,70092.500000,9.900000,30.000000,5.000000,4.4000
75%,1500.250000,54.000000,29.000000,27.000000,2.000000,4.200000,7.500000,7.700000,7.800000,83592.750000,14.900000,37.000000,6.000000,5.9000
max,2000.000000,64.000000,39.000000,53.000000,7.000000,5.000000,10.000000,10.000000,10.000000,132801.000000,20.000000,57.000000,17.000000,10.0000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   employee_id              2000 non-null   int64  
 1   age                      2000 non-null   int64  
 2   years_experience         2000 non-null   int64  
 3   education_level          2000 non-null   object 
 4   department               2000 non-null   object 
 5   company_size             2000 non-null   object 
 6   region                   2000 non-null   object 
 7   gender                   2000 non-null   object 
 8   ethnicity                2000 non-null   object 
 9   marital_status           2000 non-null   object 
 10  remote_status            2000 non-null   object 
 11  training_hours           2000 non-null   float64
 12  certifications_count     2000 non-null   int64  
 13  job_level                2000 non-null   object 
 14  performance_rating      

,employee_id,age,years_experience,education_level,department,company_size,region,gender,ethnicity,marital_status,...,manager_feedback_score,work_life_balance_score,salary_satisfaction,annual_income_usd,bonus_percent,commute_time_minutes,overtime_hours_monthly,job_satisfaction_score,promoted_next_year,left_company
0,1,59,18,bachelor,HR,small,south,female,hispanic,married,...,1.9,8.4,9.7,58066.0,1.7,33.0,9,6.5,no,no
1,2,49,10,bachelor,IT,medium,north,female,other,married,...,8.8,6.2,9.4,91402.0,13.5,30.0,5,6.6,no,no
2,3,35,0,bachelor,Finance,small,north,female,black,married,...,3.7,5.2,5.3,89372.0,11.6,40.0,5,5.4,no,no
3,4,63,23,bachelor,HR,medium,west,male,other,married,...,7.3,6.8,7.8,54380.0,5.8,48.0,2,4.2,no,no
4,5,28,6,bachelor,Finance,small,west,female,white,single,...,8.4,5.2,7.0,70198.0,15.6,37.0,6,2.5,no,yes


In [13]:
# Variance Threshold

# Select only numeric columns
numeric_df = df.select_dtypes(include=[np.number])

# Remove features with variance < 0.01
var_thresh = VarianceThreshold(threshold=0.01)
var_thresh.fit(numeric_df)

low_variance_features = numeric_df.columns[~var_thresh.get_support()]
print("Low variance features:", low_variance_features.tolist())

Low variance features: []


In [15]:
# Chi Squared Test

# Encode categorical variables numerically
cat_df = df.select_dtypes(include='object').apply(LabelEncoder().fit_transform)

# Encode target
target = LabelEncoder().fit_transform(df['left_company'])

# Apply Chi-square test
chi_scores, p_values = chi2(cat_df, target)

chi_results = pd.DataFrame({
    'feature': cat_df.columns,
    'chi2': chi_scores,
    'p_value': p_values
}).sort_values('p_value')

display(chi_results)


,feature,chi2,p_value
10,left_company,858.000000,1.325437e-188
9,promoted_next_year,3.563878,5.904965e-02
5,ethnicity,3.000692,8.322894e-02
8,job_level,1.387402,2.388444e-01
1,department,0.831565,3.618203e-01
4,gender,0.690467,4.060057e-01
6,marital_status,0.362329,5.472154e-01
2,company_size,0.281476,5.957357e-01
7,remote_status,0.190709,6.623273e-01
0,education_level,0.047526,8.274259e-01


In [ ]:
# One Way ANOVA F-Test

# Scale numeric data
X_num = MinMaxScaler().fit_transform(df.select_dtypes(include=np.number))
y = LabelEncoder().fit_transform(df['left_company'])

f_scores, p_vals = f_classif(X_num, y)

anova_results = pd.DataFrame({
    'feature': df.select_dtypes(include=np.number).columns,
    'F_score': f_scores,
    'p_value': p_vals
}).sort_values('p_value')

# Higher F_score and lower p_value = stronger relationship between numeric feature and target
display(anova_results)

,feature,F_score,p_value
13,job_satisfaction_score,1585.145754,1.030601e-255
7,work_life_balance_score,984.610025,4.621960e-176
6,manager_feedback_score,36.935288,1.458600e-09
8,salary_satisfaction,24.807738,6.878657e-07
11,commute_time_minutes,11.524011,7.004582e-04
3,training_hours,2.944036,8.635013e-02
4,certifications_count,1.487078,2.228144e-01
2,years_experience,1.306123,2.532347e-01
9,annual_income_usd,0.942828,3.316690e-01
1,age,0.924431,3.364306e-01


In [18]:
# Correlation and Mutual Information

corr = df.corr(numeric_only=True)['annual_income_usd'].sort_values(ascending=False)
display(corr)

# Encode categorical columns
X = pd.get_dummies(df.drop(columns=['left_company']))
y = LabelEncoder().fit_transform(df['left_company'])

mi = mutual_info_classif(X, y, discrete_features='auto')
mi_results = pd.Series(mi, index=X.columns).sort_values(ascending=False)

print(mi_results.head(10))

annual_income_usd          1.000000
performance_rating         0.024109
work_life_balance_score    0.021386
age                        0.017088
manager_feedback_score    -0.003874
years_experience          -0.004255
employee_id               -0.008201
bonus_percent             -0.011552
certifications_count      -0.014268
commute_time_minutes      -0.020349
training_hours            -0.029361
job_satisfaction_score    -0.036008
overtime_hours_monthly    -0.036659
salary_satisfaction       -0.057906
Name: annual_income_usd, dtype: float64

job_satisfaction_score     0.374155
work_life_balance_score    0.250089
manager_feedback_score     0.020286
department_Sales           0.019117
annual_income_usd          0.017415
certifications_count       0.017232
marital_status_divorced    0.016443
ethnicity_asian            0.016311
education_level_master     0.007860
marital_status_married     0.007035
dtype: float64
